In [11]:
import csv
import pandas as pd
import os
import re

import math
from tqdm import tqdm
import multiprocessing as mp
import time
import sched
import ast
import collections
import datetime


import numpy as np
from itertools import groupby
from sklearn.feature_extraction import text
import nltk
from nltk.corpus import stopwords


folder_prefix = '/home/jovyan/work/'

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

replace unicode spaces with actual 

bunch of junk words in text in WEBTEXT, 

using word_embedding script and the expanded stop word list on yoon's notebook

create a funciton that takes in a string
	- replace unicode spaces with actual space
	- numbers, 
	- "/" "-"
	
-data exploration, bar charts, word cloud

take in a string and return a 


Look at ping on basecamp
re.split('\W+|_', p)

In [3]:
cleaned_df = pd.read_pickle(folder_prefix + 'nowdata/charters_final_2015.pkl')

In [18]:
string = cleaned_df['CMO_WEBTEXT'][39][0][3]

In [14]:
stop_word_list = list(set(stopwords.words('english')))
#long list of manual addition to stopwords:
for i in range(1,13):
    stop_word_list.append(datetime.date(2008, i, 1).strftime('%B'))
for i in range(1,13):
    stop_word_list.append((datetime.date(2008, i, 1).strftime('%B')).lower())
for i in range(1, 2100):
    stop_word_list.append(str(i))
stop_word_list.append('00')
stop_word_list.append('el')
stop_word_list.append('en')
stop_word_list.append('la')
stop_word_list.append('los')
stop_word_list.append('para')
stop_word_list.append('las')
stop_word_list.append('san')
stop_word_list.append('mr')
stop_word_list.append('mrs')
stop_word_list.append('sa')
stop_word_list.append('angeles')
stop_word_list.append('diego')
stop_word_list.append('california')
stop_word_list.append('york')
stop_word_list.append('fax')
stop_word_list.append('email')
stop_word_list.append('phone')
stop_word_list.append('harlem')
stop_word_list.append('bronx')
stop_word_list.append('wi')
stop_word_list.append('pm')
stop_word_list.append('org')
stop_word_list.append('city')
stop_word_list.append('austin')
stop_word_list.append('antonio')
stop_word_list.append('texas')
#adding states abbreviations for stopwords
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV"]
for state in states:
          stop_word_list.append(state)
for state in [state.lower() for state in states]:
          stop_word_list.append(state)
stop_words_extra = text.ENGLISH_STOP_WORDS.union(stop_word_list) #we have text.English stopwords and nltk's english stop words

In [68]:
unicode_list  = []
for i in range(1000,1100):
    unicode_list.append(chr(i))

In [96]:
def clean_string(string, stop_words_extra, unicode_list): #I don't split up a string into words
    s = re.sub(r"\\x.*|\\u.*|\\b.*|-|\u2605$", "", string.replace(u"\xa0", u" ").replace(u"\\t", u" ").strip(" "))
    
    #s = ''.join([c for c in s if c not in unicode_list])
    #s = s.encode('ascii', errors='ignore').encode("utf-8")
    s = re.sub(r'[^\x00-\x7f]',r'', s) #got rid of hex character like \xa0\ adn a\x80
    s = re.sub(r'|'.join(map(re.escape, unicode_list)), '', s) #should remove unicode unicode
    s = s.encode("utf-8").decode("utf-8").replace(u"\u2022", "*")
    li_text = []
    for word in re.split('\n|\t|\*|\r', s): #'\W+|_' removes punctuation which we don't want
        if ((word not in (stop_words_extra)) and (not s.isdigit())):
            li_text.append(word)
    s = ' '.join(li_text)
    return s

In [97]:
clean_string(string, stop_words_extra,unicode_list ) #is the lower_case ok?
#also a lot of menu header, are broken apart, so we have things like 
#menu home cas faqs school calendar breakfast lunch menus school wellness policy 
#so there isn't an idea of sentence here

'     Center for Academic Success:  K12     Higher Standards.  Higher Expectations. Menu Home About CAS FAQs School Calendar 201819 Breakfast & Lunch Menus School Wellness Policy Parents Right to KnowTeacher Qulifications Proposed Budget FY 2019 Budget Adoption Hearing Notice Sierra Vista Campus SV Elementary School SV Middle School SV High School A+ Anywhere Login SV Administration Douglas Campus Douglas Elementary School Douglas Middle School Douglas High School Douglas Administration Board of Directors Forms & Resources Parent/Student Handbook  Elementary/Middle School Parent/Student Handbook  High School Transcript / Diploma Request New Student Applications Returning Student Application PowerSchool Login Tax Credit 2017 Before & After Care Clubs High School Club Schedule CyberPatriots Solar GoKart Team Employment Apply Now Contact Us     Higher Standards.  Higher Expectations.    Get Ready! School Starts August 8th! Mark your calendars for the first day of school on Wednesday, Augu

In [98]:
string 

'\t\t\t\t\tCenter for Academic Success:  K-12\t\t\t\t\nHigher Standards.  Higher Expectations.\nMenu\nHome\nAbout CAS\nFAQs\nSchool Calendar 2018-19\nBreakfast & Lunch Menus\nSchool Wellness Policy\nParentâ\x80\x99s Right to Know-Teacher Qulifications\nProposed Budget FY 2019\nBudget Adoption Hearing Notice\nSierra Vista Campus\nSV Elementary School\nSV Middle School\nSV High School\nA+ Anywhere Login\nSV Administration\nDouglas Campus\nDouglas Elementary School\nDouglas Middle School\nDouglas High School\nDouglas Administration\nBoard of Directors\nForms & Resources\nParent/Student Handbook â\x80\x93 Elementary/Middle School\nParent/Student Handbook â\x80\x93 High School\nTranscript / Diploma Request\nNew Student Applications\nReturning Student Application\nPowerSchool Login\nTax Credit 2017\nBefore & After Care Clubs\nHigh School Club Schedule\nCyberPatriots\nSolar Go-Kart Team\nEmployment\nApply Now\nContact Us\n\t\t\t\tHigher Standards.  Higher Expectations.\t\t\t\nGet Ready! Schoo

In [99]:
clean_string(cleaned_df['CMO_WEBTEXT'][123][0][3], stop_words_extra, unicode_list ) #is the lower_case ok?

'Email:  ccenrollment@bfcsaz.com  Phone: (480) 9870722 Gilbert Campus  Val Vista & Ray Email:  gcenrollment@bfcsaz.com  Phone: (480) 6320722 Power Campus  Power & Chandler Heights Email:  pcenrollment@bfcsaz.com  Phone: (480) 6778400 BFHS  Germann & Rittenhouse Email:  hsenrollment@bfcsaz.com  Phone: (480) 5581197         Enroll Today!     Follow these easy steps to start the process to be part of the Benjamin Franklin family: Complete this PreEnrollment Application Wait for your chosen campus(es) to contact you. Explore our website if you havent already! Please note that this application is for the PreEnrollment process only. Completion of this form does not guarantee enrollment. Quick Links       Home School News Calendar Links for Parents   Family Link JH/HS Parent Handbook K6 Parent Handbook What is a Charter School? Our Stories Enrollment Information Schedule a Tour Charger Voice Athletics Photo Galleries Employment Contact Us News & Announcements Subscribe Summer Break! Read More

In [100]:
cleaned_df['CMO_WEBTEXT'][123][0][3]

'Email: \nccenrollment@bfcsaz.com\n\rPhone: (480) 987-0722\nGilbert Campus\n\rVal Vista & Ray\nEmail: \ngcenrollment@bfcsaz.com\n\rPhone: (480) 632-0722\nPower Campus\n\rPower & Chandler Heights\nEmail: \npcenrollment@bfcsaz.com\n\rPhone: (480) 677-8400\nBFHS\n\rGermann & Rittenhouse\nEmail: \nhsenrollment@bfcsaz.com\n\rPhone: (480) 558-1197\n        Enroll Today!    \nFollow these easy steps to start the process to be part of the Benjamin Franklin family:\nComplete this\xa0Pre-Enrollment Application\nWait for your chosen campus(es) to contact you.\nExplore our website if you havenâ\x80\x99t already!\nPlease note that this application is for the Pre-Enrollment process only. Completion of this form does not guarantee enrollment.\nQuick Links\r\t\t\t\t\t\nHome\nSchool News\nCalendar\nLinks for Parents  \nFamily Link\nJH/HS Parent Handbook\nK-6 Parent Handbook\nWhat is a Charter School?\nOur Stories\nEnrollment Information\nSchedule a Tour\nCharger Voice\nAthletics\nPhoto Galleries\nEmplo

In [102]:
clean_string(cleaned_df['CMO_WEBTEXT'][905][0][3], stop_words_extra, unicode_list) #is the lower_case ok?

'icopa Survey Leading Edge Academy Online Survey Enroll Now! Click here to Enroll Leading Edge Academy is a network of K12 Tuitionfree Charter Schools with campuses throughout the valley.  Were proud to offer advanced academics and character development in a technologyenriched environment. Please have a look around to learn about the various campuses and programs that are a part of the Leading Edge Academy Network! A glimpse of what youll find:   Tuition Free   K12   Small Class Sizes   Advanced Academics   Free Full Day Kindergarten   Gilbert, Mesa, Queen Creek, and Maricopa Campuses   Online Academy In accordance with Federal Law, Leading Edge Academy does not discriminate on the basis of race, religion, color, national origin, sex, or disability.                  2018 Leading Edge Academy         Home Staff Enrollment Contact Employment'